In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
ls

gdrive/  sample_data/


In [4]:
cd sample_data/

/content/sample_data


In [5]:
!ls

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [9]:
cd ..

/content


In [10]:
cd ..

/


In [11]:
!ls

bin	 datalab  home	 lib64	opt   run   swift	       tmp    var
boot	 dev	  lib	 media	proc  sbin  sys		       tools
content  etc	  lib32  mnt	root  srv   tensorflow-1.15.2  usr


In [12]:
cd /content

/content


In [13]:
!ls

drive  gdrive  sample_data


In [15]:
cd drive

/content/drive


In [16]:
!ls

'My Drive'


In [17]:
cd 'My Drive'

/content/drive/My Drive


In [18]:
!ls

'Colab Notebooks'	      object_pose_estimation
 CrazyLegs_SuzieQ_2019-2020   Untitled


In [19]:
cd object_pose_estimation/

/content/drive/My Drive/object_pose_estimation


In [20]:
!ls

camera_calibration.ipynb  Classes  data  Untitled0.ipynb
